# LlamaParser
 
LlamaParse는 LlamaIndex에서 개발한 문서 파싱 서비스로, 대규모 언어 모델(LLM)을 위해 특별히 설계되었습니다. 주요 특징은 다음과 같습니다:

- PDF, Word, PowerPoint, Excel 등 다양한 문서 형식 지원
- 자연어 지시를 통한 맞춤형 출력 형식 제공
- 복잡한 표와 이미지 추출 기능
- JSON 모드 지원
- 외국어 지원

![alt text](image.png)

LlamaParse는 독립형 API로 제공되며, LlamaCloud 플랫폼의 일부로도 사용 가능합니다. 이 서비스는 문서를 파싱하고 정제하여 검색 증강 생성(RAG) 등 LLM 기반 애플리케이션의 성능을 향상시키는 것을 목표로 합니다.

사용자는 무료로 한달에 10,000페이지를 처리할 수 있으며, 유료 플랜을 통해 추가 용량을 확보할 수 있습니다. LlamaParse는 현재 공개 베타 버전으로 제공되고 있으며, 지속적으로 기능이 확장되고 있습니다.

- 링크: https://cloud.llamaindex.ai

**API 키 설정**
- API 키를 발급 후 `.env` 파일에 `LLAMA_CLOUD_API_KEY` 에 설정합니다.

In [ ]:
# 설치
# !pip install llama-index-core llama-parse llama-index-readers-file python-dotenv

In [2]:
import os
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()
# jupyter 환경에서 asyncio를 사용할 수 있도록 설정
# 이는 Jupyter Notebook에서 비동기 작업을 지원하기 위해 필요합니다.
nest_asyncio.apply()

기본 파서 적용

In [ ]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# 파서 설정
parser = LlamaParse(
    result_type="markdown",  # "markdown"과 "text" 사용 가능
    num_workers=8,  # worker 수 (기본값: 4)
    verbose=True,
    language="ko",
)

# SimpleDirectoryReader를 사용하여 파일 파싱
file_extractor = {".pdf": parser}

# LlamaParse로 파싱하고자하는 파일 목록을 input_files에 지정합니다.
documents = SimpleDirectoryReader(
    input_files=["data/SPRI_AI_Brief_2023년12월호_F.pdf"],
    file_extractor=file_extractor,
).load_data()

Started parsing the file under job_id 0c29b0be-a687-42f1-b7bf-2805f04a458c
.

In [19]:
# 페이지 수 확인
len(documents)

16

In [4]:
documents[0]

Document(id_='128451be-2864-49f0-a173-4a28fc5ac9bd', embedding=None, metadata={'file_path': 'data\\SPRI_AI_Brief_2023년12월호_F.pdf', 'file_name': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'file_type': 'application/pdf', 'file_size': 975735, 'creation_date': '2025-08-21', 'last_modified_date': '2025-08-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='# 인공지능 산업의 최신 동향\n\n# 2023년 12월호\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

LlamaIndex -> LangChain Document 로 변환

In [20]:
# 랭체인 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in documents]

In [21]:
print(docs[5].page_content)




6  Z. Shen et al.

Fig. 2: The relationship between the three types of layout data structures. Coordinate supports three kinds of variation; TextBlock consists of the coordinate information and extra features like block text, types, and reading orders; a Layout object is a list of all possible layout elements, including other Layout objects. They all support the same set of transformation and operation APIs for maximum flexibility.

Shown in Table 1, LayoutParser currently hosts 9 pre-trained models trained on 5 different datasets. Description of the training dataset is provided alongside with the trained models such that users can quickly identify the most suitable models for their tasks. Additionally, when such a model is not readily available, LayoutParser also supports training customized layout models and community sharing of the models (detailed in Section 3.5).

# 3.2  Layout Data Structures

A critical feature of LayoutParser is the implementation of a series of data structu

In [22]:
# metadata 출력
docs[0].metadata

{'file_path': 'data\\2103.15348v2.pdf',
 'file_name': '2103.15348v2.pdf',
 'file_type': 'application/pdf',
 'file_size': 4686220,
 'creation_date': '2025-08-22',
 'last_modified_date': '2025-08-22'}

## MultiModal Model 로 파싱

**주요 파라미터**

- `use_vendor_multimodal_model`: 멀티모달 모델 사용 여부를 지정합니다. `True`로 설정하면 외부 벤더의 멀티모달 모델을 사용합니다.

- `vendor_multimodal_model_name`: 사용할 멀티모달 모델의 이름을 지정합니다. 여기서는 "openai-gpt4o"를 사용하고 있습니다.

- `vendor_multimodal_api_key`: 멀티모달 모델 API 키를 지정합니다. 환경 변수에서 OpenAI API 키를 가져옵니다.

- `result_type`: 파싱 결과의 형식을 지정합니다. "markdown"으로 설정되어 있어 결과가 마크다운 형식으로 반환됩니다.

- `language`: 파싱할 문서의 언어를 지정합니다. "ko"로 설정되어 한국어로 처리됩니다.

- `skip_diagonal_text`: 대각선 텍스트를 건너뛸지 여부를 결정합니다.

- `page_separator`: 페이지 구분자를 지정할 수 있습니다.

In [ ]:
GEMINI = False
if GEMINI:
    multimodal_model_name = "gemini-2.5-pro"
    multimodal_api_key = os.environ["GEMINI_API_KEY"]
else:
    multimodal_model_name = "openai-gpt4o"
    multimodal_api_key = os.environ["OPENAI_API_KEY"]

    # LlamaParse 설정

documents = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name=multimodal_model_name,
    vendor_multimodal_api_key=multimodal_api_key,
    result_type="markdown",
    language="ko",
    # skip_diagonal_text=True,
    # page_separator="\n=================\n"
)

In [ ]:
# parsing 된 결과
# file_path = "data/SPRI_AI_Brief_2023년12월호_F.pdf"
file_path = "data/2103.15348v2.pdf"
parsed_docs = documents.load_data(file_path=file_path)

Started parsing the file under job_id b50de87a-8447-4ad7-bb79-3bd4e513838f


In [34]:
# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

In [ ]:
import os

file_path = "data/2103.15348v2.pdf"
parsed_docs = documents.load_data(file_path=file_path)

# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

# os.path.splitext()를 사용하여 경로와 확장자를 분리하고, 새로운 확장자를 붙입니다.
# file_root는 'data/2103.15348v2'가 됩니다.
file_root, _ = os.path.splitext(file_path)
output_file_path = file_root + ".md"

# 1. 모든 페이지의 page_content를 리스트로 추출합니다.
#    각 페이지 사이를 두 줄 띄어쓰기(\n\n)로 구분하여 가독성을 높입니다.
full_text = "\n\n".join([doc.page_content for doc in docs])

# 2. 추출한 전체 텍스트를 파일에 저장합니다.
#    'w' 모드는 파일을 쓰기 모드로 열며, encoding='utf-8'은 한글 깨짐을 방지합니다.
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write(full_text)

print(f"✅ 파일 저장 완료: {output_file_path}")

✅ 파일 저장 완료: data/2103.15348v2.md


In [ ]:
print(docs[3].page_content



1. 정책/정책

# 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표

**KEY Contents**

- 미국 바이든 대통령이 '안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령'에 서명하고 광범위한 행정 조치를 명시
- 행정명령은 AI의 안전과 보안 기준 마련, 개인정보보호, 소행성화와 시민권 향상, 소비자 보호, 노동자 지원, 습식내 경쟁 촉진, 스크재협력을 골자로 함

## 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진

- 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표
- 행정명령은 AI의 안전과 보안 기준 마련, 개인정보보호, 소행성화와 시민권 향상, 소비자 보호, 노동자 지원, 습식내 경쟁 촉진, 스크재협력에 관한 내용을 포함
- (AI 안전과 보안 기준) 강화된 AI 시스템을 개발하는 기업에게 안전 테스트 결과와 시스템에 관한 주요 정보를 미국 정부와 공유할 것을 요구하고, AI 시스템의 안전성과 신뢰성 확인을 위한 표준 및 AI 생성 콘텐츠 표시를 위한 표준과 모바일 AI 활용을 추진
- 10<sup>26</sup> 플롭스(FLOPS, Floating Point Operation Per Second)를 초과하는 컴퓨터 성능 또는 상용화적 서열 데이터를 주로 사용하는 10<sup>3</sup>플롭스를 초과하는 컴퓨터 성능을 사용하는 모델 스탠딩 데이터센터에서 1,000기가 이상의 네트워크로 연결되며 AI 훈련에서 이로한 최대 10<sup>20</sup> 플롭스를 처리할 수 있는 컴퓨터 용량을 갖춘 클러스터가 정보공유 요구대상
- (행동성과 시민권 향상) 법률, 주택, 보건 분야에서 AI의 무책임한 사용으로 인한 차별과 편견 및 기타 문제를 방지하는 조치를 확대
- 형사사법 시스템에서 AI 사용 모범사례를 개발하고, 주택 임대 시 AI 알고리즘 차별을 막기 위한 명확한 지침을 제공하며

아래와 같이 사용자 정의 인스트럭션을 지정하는 것도 가능합니다.

In [ ]:
# parsing instruction 을 지정합니다.
parsing_instruction = (
    "You are parsing a AI Report. Please extract tables in markdown format."
)

# LlamaParse 설정
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
    result_type="markdown",
    language="ko",
    parsing_instruction=parsing_instruction,
)

# parsing 된 결과
parsed_docs = parser.load_data(file_path=file_path)

# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

In [ ]:
# markdown 형식으로 추출된 테이블 확인
print(docs[-2].page_content)

### LLAMA_PARSER를 함수 실행

In [4]:
import os
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

documents = LlamaParse(result_type="markdown")


def pdf_parser(pdf_file_path: str):
    """
    PDF 파일을 파싱하여 그 내용을 Markdown 파일로 저장합니다.

    Args:
        pdf_file_path (str): 처리할 PDF 파일의 경로.
    """
    print(f"🔄 '{pdf_file_path}' 파일 파싱을 시작합니다...")

    try:
        # parsing instruction 을 지정합니다.
        parsing_instruction = (
            "You are parsing a AI Report. Please extract tables in markdown format."
        )

        # LlamaParse 설정
        parser = LlamaParse(
            use_vendor_multimodal_model=True,
            vendor_multimodal_model_name="openai-gpt4o",
            vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
            result_type="markdown",
            parsing_mode="Unstructured",
            language="ko",
            parsing_instruction=parsing_instruction,
        )

        # 1. LlamaParse를 사용하여 PDF 파일을 로드합니다.
        # 'documents' 객체는 이 함수 외부에서 미리 정의되어 있어야 합니다.
        parsed_docs = documents.load_data(file_path=pdf_file_path)

        # 2. LangChain 형식의 도큐먼트로 변환합니다.
        docs = [doc.to_langchain_format() for doc in parsed_docs]

        # 3. 저장할 Markdown 파일의 경로를 생성합니다. (확장자 변경)
        file_root, _ = os.path.splitext(pdf_file_path)
        output_file_path = file_root + ".md"

        # 4. 모든 페이지의 내용을 하나의 텍스트로 합칩니다.
        #    페이지 사이는 두 줄로 띄어 가독성을 높입니다.
        full_text = "\n\n".join([doc.page_content for doc in docs])

        # 5. 추출된 전체 텍스트를 .md 파일로 저장합니다.
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(full_text)

        print(f"✅ 파일 저장 완료: {output_file_path}")

    except FileNotFoundError:
        print(f"❌ 오류: 파일을 찾을 수 없습니다 - {pdf_file_path}")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")


# --- 함수 사용 예시 ---
# 이 코드를 실행하기 전에 'documents' 파서 객체를 초기화해야 합니다.
# file_to_parse = "data/디지털정부혁신추진계획.pdf"
file_to_parse = "./data/클라우드네이티브1-2.pdf"
pdf_parser(file_to_parse)

🔄 './data/클라우드네이티브1-2.pdf' 파일 파싱을 시작합니다...
Started parsing the file under job_id 555fa74a-64fb-45be-b0d6-052addd2f4c3
✅ 파일 저장 완료: ./data/클라우드네이티브1-2.md
